In [1]:

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/app')

In [3]:

from audio.redis import *
from audio.audio import *
import io
from faster_whisper import WhisperModel

import json

import asyncio

model_size = "large-v3"
redis_client = await get_redis()

model = WhisperModel(model_size, device="cuda", compute_type="float16")

In [4]:

async def process():
    _,segment = await redis_client.brpop('TranscribeQueue')
    segment = json.loads(segment)
    audio = Audio(segment['audio_id'],redis_client)
    if await audio.get():
        print(segment['name'])
    try:
        audio_data_slicer = AudioSlicer(audio.data)
        segment_audio_data = await audio_data_slicer.export_data(segment['start'],segment['end'])

        segments, _ = model.transcribe(io.BytesIO(segment_audio_data), beam_size=5)
        data = [{'start':s.start, 'end':s.end, 'text':s.text} for s in list(segments)]
        await Transcript(segment['name'],redis_client,data).save()
        await redis_client.publish(f'TranscribeReady', segment["name"])
    except Exception as e:
        print(e)
        await redis_client.publish(f'TranscribeReady', segment["name"])



In [11]:
_,segment = await redis_client.brpop('TranscribeQueue')
segment = json.loads(segment)
audio = Audio(segment['audio_id'],redis_client)
if await audio.get():
    print(segment['name'])

audio_data_slicer = AudioSlicer(audio.data)
segment_audio_data = await audio_data_slicer.export_data(segment['start'],segment['end'])

segments, _ = model.transcribe(io.BytesIO(segment_audio_data), beam_size=5)
data = [{'start':s.start, 'end':s.end, 'text':s.text} for s in list(segments)]
await Transcript(segment['name'],redis_client,data).save()
await redis_client.publish(f'TranscribeReady', segment["name"])


In [9]:

await process()


e929ea4c-2d63-45eb-90d9-2d1d6e161564
[Errno 2] No such file or directory: 'ffprobe'


/usr/local/lib/python3.10/dist-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)
